In [1]:
import sys
import os
import keras_tuner
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Layer, Dropout
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU


# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("transformer0.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from FNN1_1 import baseline_deviation, baeline_out_deviation, baseline_long_deviation, baseline_relError, absSum
baseline_out_deviation = baeline_out_deviation

from GetXY import x_train, y_train, x_val, y_val, x_test, y_test, out_x_test, out_y_test, long_x_test, long_y_test, outsideExpr, absSum, tokenizer

C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. P

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 23.4864 - val_loss: 19.1676
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 17.8426 - val_loss: 15.7216
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.0301 - val_loss: 14.5667
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.4935 - val_loss: 12.9158
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.1981 - val_loss: 10.3270
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.8995 - val_loss: 6.7563
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0913 - val_loss: 3.2757
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2923 - val_loss: 1.4743
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1752 - val_loss: 0.9445
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8619 - val_loss: 0.7479
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6855 - val_loss: 0.5979
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step -

C:\Users\A_118784\Desktop\matura_github\matura\matura\FNN1_1.py:246: RuntimeWarning: divide by zero encountered in divide
  relativeError = np.where(np.array(y_test) != 0, deviation.flatten() / np.abs(np.array(y_test)), deviation.flatten())


4 + 1 - -2
2543
7.0

Expressions not in x:
0 + 3 - 2
True
1457
1.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]


In [2]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='val_loss',
    mode = "min"
)

In [3]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [4]:
input_shape = x_train[0].shape
from tensorflow.keras import backend as K

def build_model(hp, input_shape):
    K.clear_session()
    num_neurons = hp.Int("num_neurons", 1, 512)
    num_layers = hp.Int("num_layers", 1, 16)
    dropoutTF = hp.Boolean("dropoutT/F")
    lil_model = keras.Sequential()
    lil_model.add(keras.Input(shape=input_shape))
    lil_model.add(layers.Flatten())
    for i in range(num_layers):
        lil_model.add(layers.Dense(num_neurons)),
        lil_model.add(PReLU())
        if dropoutTF == True:
            lil_model.add(layers.Dropout(0.1))
    lil_model.add(layers.Dense(1, activation='linear'))
    lil_model.compile(optimizer="adam", loss="mse")
    return lil_model
build_model(keras_tuner.HyperParameters(), input_shape)

<Sequential name=sequential, built=True>

In [5]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=lambda hp: build_model(hp, input_shape),    objective="val_loss",
    max_trials=100,
    executions_per_trial=1,
    overwrite=False,
    directory="FNNTuner",
    project_name="tuner",
)

Reloading Tuner from FNNTuner\tuner\tuner0.json


In [6]:
num_epochs = 100
tuner.search(train_dataset, epochs = num_epochs, validation_data = (val_dataset), verbose = 1, callbacks = [])

In [7]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hps, input_shape)
best_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 15)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 345)                 │           5,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ p_re_lu (PReLU)                      │ (None, 345)                 │             345 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             346 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,211 (24.26 KB)

 Trainable params: 6,211 (24.26 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
x_test_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
out_x_test_dataset = tf.data.Dataset.from_tensor_slices(out_x_test).batch(batch_size)
long_x_test_dataset = tf.data.Dataset.from_tensor_slices(long_x_test).batch(batch_size)

In [9]:
best_model.load_weights("FNN2_weights.weights.h5")

C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [10]:
predsInRange = best_model.predict(x_test_dataset)
predsOutRange = best_model.predict(out_x_test_dataset)
predsLongRange = best_model.predict(long_x_test_dataset)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [11]:
import numpy as np

reldiffInRange = []
diffInRange = []
safe_y_test = np.where(np.isclose(y_test,0.0), 1.0, y_test)

for i in range(len(y_test)):
    diffInRange.append(abs(y_test[i] - predsInRange[i]))
    reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
print(len(diffInRange))
print("MAE in Range: ", np.mean(diffInRange))
print("MRE in Range: ", np.mean(reldiffInRange))

diffLongRange = []
for i in range(200, 300):
    diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
    
NEEDdiffLongRange = []
for i in range(len(long_y_test)):
    NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
print("MAE longer Expressions: ", np.mean(NEEDdiffLongRange))

diffOutRange = []
for i in range(len(out_y_test)):
    diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
diff_out_relError = []
for i in range(len(out_y_test)):
    diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
print("MAE out Range: ", np.mean(diffOutRange))
print("MRE out Range: ", np.mean(diff_out_relError))

1457
MAE in Range:  0.030438986
MRE in Range:  0.012106994744889207
MAE longer Expressions:  6.508988744705691
MAE out Range:  2.2662077
MRE out Range:  0.2579330530220375


In [12]:
placeholder = absSum(outsideExpr)
diffOutRange = []
indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 

for i in indices_with_placeholder_22:
    diffOutRange.append(np.abs(out_y_test[i]-predsOutRange[i]))


In [13]:
meanDiff_InRange = np.mean(diffInRange)
meanDiff_OutRange = np.mean(diffOutRange)
meanDiff_LongRange = np.mean(diffLongRange)
meanDiff_OutRelRange = np.mean(diff_out_relError)



In [14]:
benchmark = 0
benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
print(baseline_deviation / (meanDiff_InRange**2) / 4)

benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
print(baseline_out_deviation / (meanDiff_OutRange**2) / 4)

benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
print(baseline_long_deviation / (meanDiff_LongRange**2) / 4)

benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
print(baseline_relError / (meanDiff_OutRelRange**2) / 4)

print(f"Benchmark: {benchmark}")

10.574566800628952
0.25801469464853477
0.08142093441123974
0.14579046235586743
Benchmark: 11.059792892044594


In [15]:
print("---------------------------------------------------")
print("HELLO AND WELCOME TO THE NEURAL PREDICTIVE CALCULATOR - NPC")
print("---------------------------------------------------")

---------------------------------------------------
HELLO AND WELCOME TO THE NEURAL PREDICTIVE CALCULATOR - NPC
---------------------------------------------------


In [ ]:
while True:
    user_expr = input("What do you want me to calculate?: ")
    print(f"what you asked: {user_expr}")
    model_pred = best_model.predict(tokenizer([user_expr]))
    print(f"This should be the answer: {model_pred}")
    print("---------------------------------------------------")

What do you want me to calculate?:  1 + 2 + 3


what you asked: 1 + 2 + 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
This should be the answer: [[6.0542865]]
---------------------------------------------------


What do you want me to calculate?:  6 + 2 + 3


what you asked: 6 + 2 + 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
This should be the answer: [[10.843462]]
---------------------------------------------------
